In [2]:
import json
with open('./france-20230101.geojson') as f:
    data = json.load(f)

important_features = ['id_local', 'ame_d', 'regime_d', 'sens_d', 'local_d', 'statut_d', 'revet_d', 'ame_g', 'regime_g', 'sens_g', 'local_g', 'statut_g', 'revet_g',
                        'largeur_d', 'largeur_g',
                        'trafic_vit', 'lumiere',
                        'd_service', 'date_maj', 'access_ame']

useless_features = ['id_osm', 'code_com_d', 'num_iti', 'source', 'ref_geo', 'code_com_g', 'project_c']

count = {key:0 for key in important_features + useless_features}

for feature in data['features']:
    for key in feature['properties'].keys():
        count[key] += 1

    #if 'd_service' in feature['properties']:
    #    print(feature['properties'])

reduced_count = {key:count[key]/len(data['features']) for key in count.keys()}
print(dict(sorted(reduced_count.items(), key=lambda item: item[1])))



{'d_service': 0.006572543164335537, 'largeur_g': 0.04008665803014001, 'largeur_d': 0.04412679592180284, 'access_ame': 0.09581421220970658, 'num_iti': 0.11879249646853889, 'lumiere': 0.22873987220868192, 'local_g': 0.305550066237768, 'local_d': 0.3081666410498503, 'revet_d': 0.5849819584422048, 'revet_g': 0.5849856179873966, 'regime_g': 0.6879139860498137, 'sens_g': 0.801008570654839, 'trafic_vit': 0.8546209077135893, 'regime_d': 0.8604652013847719, 'code_com_g': 0.9992534527808884, 'code_com_d': 0.9992607718712718, 'sens_d': 0.9997840868336884, 'ame_d': 0.9999743831836579, 'id_local': 1.0, 'statut_d': 1.0, 'ame_g': 1.0, 'statut_g': 1.0, 'date_maj': 1.0, 'id_osm': 1.0, 'source': 1.0, 'ref_geo': 1.0, 'project_c': 1.0}


In [17]:
print(data['features'][0].keys())
print(data['features'][0]["geometry"])      #interessant de prendre en compte la longueur de la route
print(data['features'][0]["properties"])

dict_keys(['type', 'properties', 'geometry'])
{'type': 'LineString', 'coordinates': [[0.005850700000012, 47.0862118], [0.0057496, 47.08643619999999], [0.0056467, 47.0865687]]}
{'id_local': 'geovelo_429754890_86274', 'id_osm': '429754890', 'code_com_d': '86274', 'ame_d': 'AUTRE', 'regime_d': 'EN AGGLOMERATION', 'sens_d': 'UNIDIRECTIONNEL', 'statut_d': 'EN SERVICE', 'revet_d': 'LISSE', 'code_com_g': '86274', 'ame_g': 'AUTRE', 'regime_g': 'EN AGGLOMERATION', 'sens_g': 'UNIDIRECTIONNEL', 'statut_g': 'EN SERVICE', 'revet_g': 'LISSE', 'date_maj': '2020-07-28', 'trafic_vit': 30, 'source': 'Les contributeurs OpenStreetmap', 'project_c': '4326', 'ref_geo': 'OpenStreetmap'}


In [21]:
def get_feature_repartition(feat, verbose=True):
    res = {}
    n_missing = 0
    for elem in data['features']:
        if feat not in elem["properties"]:
            n_missing += 1
            continue
        if elem["properties"][feat] not in res:
            res[elem["properties"][feat]] = 1
        else:
            res[elem["properties"][feat]] += 1
    
    missing_rate = n_missing/len(data['features'])
    if verbose:
        print(f"Feature {feat}: missing values account for {100*missing_rate:.2f}% of the dataset")
    return res, missing_rate
from math import radians, cos, sin, asin, sqrt

def distance(lon1, lat1, lon2, lat2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    return(c * r)
    
def get_feature_len(feat, verbose=True):
    def get_len(coords):
        d=0
        for i in range(1, len(coords)):
            d += distance(*coords[i-1], *coords[i])
        return d
    res = {}
    n_missing = 0
    missing_dist = 0
    for elem in data['features']:
        if feat not in elem["properties"]:
            n_missing += 1
            missing_dist += get_len(elem["geometry"]["coordinates"])
            continue
        if elem["properties"][feat] not in res:
            res[elem["properties"][feat]] = get_len(elem["geometry"]["coordinates"])
        else:
            res[elem["properties"][feat]] += get_len(elem["geometry"]["coordinates"])
    
    missing_rate = n_missing/len(data['features'])
    total_dist = sum(res.values()) + missing_dist
    if verbose:
        print(f"Feature {feat}: missing values account for {100*missing_rate:.2f}% of the dataset, ({100*missing_dist/total_dist:.2f}% of the routes length missing)")
    return res, missing_rate

In [20]:
t = ["ame_d", "ame_g", "regime_d", "regime_g", "sens_d", "sens_g", "local_g", "local_d", "trafic_vit"]
u = ["id_local", "statut_d", "statut_g", "revet_d", "revet_g", "d_service", "date_maj"]
for feat in important_features:
    if feat in t+u:
        continue
    #repartition, missing_rate = get_feature_repartition(feat)
    repartition, missing_rate = get_feature_len(feat)
    if len(repartition) <= 20 and missing_rate < .7:
        print(repartition)


Feature largeur_d: missing values account for 95.59% of the dataset, (92.60% of the routes lenght missing)
Feature largeur_g: missing values account for 95.99% of the dataset, (92.75% of the routes lenght missing)
Feature lumiere: missing values account for 77.13% of the dataset, (83.65% of the routes lenght missing)
Feature access_ame: missing values account for 90.42% of the dataset, (82.31% of the routes lenght missing)


In [31]:
rep_g, _ = get_feature_len("ame_g")
rep_d, _ = get_feature_len("ame_d")

key_cyclables = ["PISTE CYCLABLE", "BANDE CYCLABLE", "DOUBLE SENS CYCLABLE BANDE", "DOUBLE SENS CYCLABLE PISTE"]
key_non_cyclables = ["AUCUN", "CHAUSSEE A VOIE CENTRALE BANALISEE", "VELO RUE"]

key_sens_unique = ["PISTE CYCLABLE", "BANDE CYCLABLE"]
key_sens_double = ["DOUBLE SENS CYCLABLE BANDE", "DOUBLE SENS CYCLABLE PISTE"]

rep_cyclable = {"cyclable":0, "non cyclable":0}
rep_sens = {"unique":0, "double":0}

for key in key_cyclables + key_non_cyclables:
    if key in key_cyclables:
        rep_cyclable["cyclable"] += rep_g[key] + rep_d[key]
    else:
        rep_cyclable["non cyclable"] += rep_g[key] + rep_d[key]
    if key in key_sens_unique:
        rep_sens["unique"] += rep_g[key] + rep_d[key]
    elif key in key_sens_double:
        rep_sens["double"] += rep_g[key] + rep_d[key]

print(rep_cyclable) #reserve aux velos ou non
print(rep_sens)



Feature ame_g: missing values account for 0.00% of the dataset, (0.00% of the routes length missing)
Feature ame_d: missing values account for 0.00% of the dataset, (0.00% of the routes length missing)
{'cyclable': 50954.11069513227, 'non cyclable': 15069.15925638992}
{'unique': 49770.44284491312, 'double': 1183.6678502191487}


In [32]:
rep_g, _ = get_feature_len("local_g")
rep_d, _ = get_feature_len("local_d")
rep = {}
for key in rep_g:
    rep[key] = rep_g[key] + rep_d[key]

print(rep)


Feature local_g: missing values account for 69.44% of the dataset, (74.75% of the routes length missing)
Feature local_d: missing values account for 69.18% of the dataset, (76.71% of the routes length missing)
{'CHAUSSEE': 22970.042944178083, 'TROTTOIR': 2935.9675154151882}
